In [1]:
#2:40 per month
import pandas as pd
import argparse
import numpy as np
from Library.Cal_stats_lib.utils.utils import *
import dotmap
from datetime import datetime as dt
from Library.BLRPRmodel.BLRPRx import *

from Library.Fitting_lib import fitting, objectiveFunction
from datetime import datetime

args = yaml.load(open('./config/default.yaml'), Loader=yaml.FullLoader)
args = dotmap.DotMap(args)

# step 0 read statistical properties and corresponding weights
num_month = pd.read_csv(args.IO.stats_file_path, index_col=0, header=0).shape[0]
TSF = pd.read_csv(args.IO.config_path, index_col=0, header=None)
timeScale = TSF.loc['time'].dropna().tolist()
timeScaleList = fitting.scalesTransform(timeScale)
print('There are {} rows to fit'.format(num_month))
file = pd.read_csv('./02 Output_data/result.csv')
result_csv = []
sav = {}


# for each calendar month
for month in range(1,  num_month+1):
    print(f'====================== Month : {month} =======================')
    # step 1 set initial theta
    theta = [None] * 9
    if args.fitting.theta_initial is None:
        theta[0] = 0.01  # lambda #
        theta[1] = 0.1  # iota #
        theta[2] = 1.0  # sigma/mu
        theta[3] = 1e-6  # spare space
        theta[4] = 2.1  # alpha #
        theta[5] = 7.1  # alpha/nu
        theta[6] = 0.1  # kappa #
        theta[7] = 0.1  # phi #
        theta[8] = 1.0  # c
    else:
        theta = args.fitting.theta_initial
    #print(f'Original theta : {theta}')

    # step 2 initialize objective function
    obj = objectiveFunction.Exponential_func

    # step 3 initialize fitting model
    if args.fitting.moment == 'BLRPRx':
        fitting_model = BLRPRx(args.fitting.intensity_mode)

    # step 4 find approximate global optimum using Dual Annealing algorithm.
    past = datetime.now()
    final, score = fitting.Annealing(
        theta, obj, fitting_model, month, timeScaleList, args.IO.stats_file_path, args.IO.weight_file_path, args.IO.config_path)
    
    print('Time cost == {}, Row {} theta after Annealing == {}'.format((datetime.now() - past), month,
                                                                       ["%.9f" % elem for elem in final]))
    if score >= 1:
        past = datetime.now()
        final, score = fitting.Basinhopping(theta, obj, fitting_model, month, timeScaleList,
                                            args.IO.stats_file_path, args.IO.weight_file_path, args.IO.config_path)
        print('Time cost == {}, Row {} theta after Basinhopping == {}'.format((datetime.now() - past),
                                                                              month,["%.9f" % elem for elem in final]))
    
    # step 5 try to find a better local minimum using Nelder Mead algorithm
    past = datetime.now()
    local_final, local_score = fitting.Nelder_Mead(
        final, obj, fitting_model, month, timeScaleList, args.IO.stats_file_path, args.IO.weight_file_path, args.IO.config_path)
    
    print('Time cost == {}, Row {} theta after Nelder_Mead == {}'.format((datetime.now() - past), month,
                                                                         ["%.9f" % elem for elem in local_final]))

    final = local_final if local_score < score else final
    final_stats = objectiveFunction.Cal(final, timeScaleList, fitting_model)
    sav[month] = final_stats
    print(f'Final Stats : {final_stats}')
    
    # do some minor modifications
    # delete useless variables
    final = np.delete(final, [2, 3])
    final[-1] = 1.0
    result_csv.append(final)
    file.loc[month] = final
    
prop = pd.read_csv(args.IO.config_path, index_col=0, header=None)
propertyList = prop.loc['prop'].dropna().to_numpy().tolist()
    
cols = []
for sinscale in timeScale:
    for sinStaprop in propertyList[:-1]:
        cols.append(str(sinStaprop) + '_' + str(sinscale))

cols.insert(0,'Mean_1h')
# step 6 output the result theta
file.to_csv('./02 Output_data/Theta.csv', index=False)

df = pd.DataFrame(sav).T
df.columns = cols
df.to_csv('Mfinal_staProp.csv')
print('Done')

There are 12 rows to fit
====================== Month : 1 =======================
Time cost == 0:01:42.111529, Row 1 theta after Annealing == ['0.022152070', '0.446267031', '1.000000000', '0.000001000', '1.241121967', '4.458169703', '0.368427742', '0.052609615', '1.000000000']
The fval of Basinhopping = 1.143779856060076
Time cost == 0:01:22.852061, Row 1 theta after Basinhopping == ['0.022151872', '0.446255373', '0.524392904', '1.807197011', '1.241118247', '4.458321583', '0.368432981', '0.052608396', '2.857671930']


/home/user/miniconda3/envs/atm/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


The best method is trust-constr
The fval of local minimum = 1.1437798556659005
Time cost == 0:00:00.618470, Row 1 theta after Nelder_Mead == ['0.022151916', '0.446255435', '0.524415413', '1.807219131', '1.241109130', '4.458311428', '0.368431291', '0.052608459', '2.857666446']
Final Stats : [0.07911563809045719, 4.749857428733029, 0.40809664044139277, 9.852259417799925, 3.5698204852181195, 0.3312697907095644, 6.564143852915636, 2.9124898336910707, 0.2632579983060303, 5.055139665912547, 1.7838001564414683, 0.12243879028898505, 2.8787056655871597]
====================== Month : 2 =======================
Time cost == 0:01:18.146839, Row 2 theta after Annealing == ['0.006155538', '0.660764163', '1.000000000', '0.000001000', '0.566944127', '6.307934028', '0.112374573', '0.007380687', '1.000000000']
The fval of Basinhopping = 1.321400135731861
Time cost == 0:01:19.924638, Row 2 theta after Basinhopping == ['0.006155527', '0.660733636', '2.226984014', '0.000001000', '0.566923974', '6.309236092

/home/user/miniconda3/envs/atm/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


The best method is SLSQP
The fval of local minimum = 1.3214001357212999
Time cost == 0:00:01.146023, Row 2 theta after Nelder_Mead == ['0.006155526', '0.660733636', '2.226984014', '0.000001000', '0.566923974', '6.309236092', '0.112356477', '0.007379235', '0.970956378']
Final Stats : [0.06599392542432432, 5.537925782087483, 0.33857375898618786, 14.206297565518065, 4.037730063116373, 0.37036632533484165, 9.091488495785962, 3.342263010627679, 0.3839675658651117, 7.136174439403185, 2.292767104265505, 0.30230877649320614, 4.552099747928978]
====================== Month : 3 =======================
Time cost == 0:01:37.031633, Row 3 theta after Annealing == ['0.014442905', '0.541810615', '1.000000000', '0.000001000', '3.145466650', '2.030710596', '0.260940660', '0.037855919', '1.000000000']
The best method is TNC
The fval of local minimum = 0.3027010472897588
Time cost == 0:00:00.609930, Row 3 theta after Nelder_Mead == ['0.014442903', '0.541810586', '1.000000000', '0.000001000', '3.145466647

...

The best method is SLSQP
The fval of local minimum = 0.17975105622510887
Time cost == 0:00:02.172593, Row 9 theta after Nelder_Mead == ['0.012591262', '0.381809786', '1.000000000', '0.000001000', '1.044186321', '20.000000000', '0.199428735', '0.015103316', '1.000000000']
Final Stats : [0.06828670759754885, 6.143870220555906, 0.39557576323747007, 11.603824669271816, 4.608076084147218, 0.34109744802470754, 8.06650499892659, 3.773417083792199, 0.2629407125125928, 6.378607380278492, 2.3061282719185856, 0.1188328617875858, 3.675453383994356]
====================== Month : 10 =======================
Time cost == 0:01:07.189551, Row 10 theta after Annealing == ['0.010305423', '0.246838908', '1.000000000', '0.000001000', '0.563717795', '20.000000000', '0.444737704', '0.012548133', '1.000000000']


/home/user/miniconda3/envs/atm/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


The best method is L-BFGS-B
The fval of local minimum = 0.12837780332234322
Time cost == 0:00:01.620543, Row 10 theta after Nelder_Mead == ['0.010305423', '0.246838908', '1.000000000', '0.000001000', '0.563717795', '20.000000000', '0.444737704', '0.012548133', '1.000000000']
Final Stats : [0.09270178285232045, 5.2343548827005275, 0.5292237856168338, 11.929528634716311, 4.194207727951785, 0.4139797221173502, 9.017147361670204, 3.52660264517421, 0.3175903643279308, 7.18557028361532, 2.2455593899735926, 0.16285643785486165, 4.060830560028749]
====================== Month : 11 =======================
Time cost == 0:01:21.383695, Row 11 theta after Annealing == ['0.015772789', '0.410297225', '1.000000000', '0.000001000', '1.817060057', '3.343902921', '0.632824555', '0.053778156', '1.000000000']


/home/user/miniconda3/envs/atm/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


The best method is L-BFGS-B
The fval of local minimum = 0.14431139343214466
Time cost == 0:00:00.868341, Row 11 theta after Nelder_Mead == ['0.015772789', '0.410297225', '1.000000000', '0.000001000', '1.817060057', '3.343902921', '0.632824555', '0.053778156', '1.000000000']
Final Stats : [0.08262408828372021, 4.660139977080999, 0.5035732741673541, 8.898860615052154, 3.6912321569659987, 0.42721586599790207, 6.295684769642746, 3.118180098080974, 0.34835181283010097, 5.056323472630921, 2.0225179281947527, 0.15614274727100858, 3.126517160454531]
====================== Month : 12 =======================
Time cost == 0:01:27.039549, Row 12 theta after Annealing == ['0.033120998', '1.009569090', '1.000000000', '0.000001000', '3.974101390', '0.479972420', '20.000000000', '16.562847572', '1.000000000']
The fval of Basinhopping = 0.21544229116286878
Time cost == 0:01:13.822235, Row 12 theta after Basinhopping == ['0.014410740', '0.111147186', '3.149256212', '4.640282280', '1.352537082', '20.0000